# CNN and LRP on Social Physical brain

This notebook shows how saliency maps are computed for social and physical pain brain dataset. We will first train a model and then apply LRP methods. 

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.simplefilter('ignore')

In [2]:
%matplotlib inline  

import h5py
import imp
import numpy as np
import os
from sklearn.metrics import accuracy_score

import keras
import keras.backend
import keras.models

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)


from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras import regularizers
from keras import backend as K

import sys
sys.path.insert(0, '../../')

import innvestigate
import innvestigate.utils as iutils
import innvestigate.utils.visualizations as ivis

# Use utility libraries to focus on relevant iNNvestigate routines.
eutils = imp.load_source("utils", "../utils.py")
mnistutils = imp.load_source("utils_mnist", "../utils_mnist.py")





Using TensorFlow backend.


## Data
Load the dataset and keep some images from the test set for the analysis.

In [3]:
def generate_sequences(batch_size, tr_data):
   
    while True:
        """
        #this is working everytime model trained - making Batch-size data
        """
        tr_sample_count = tr_data['X_data'].shape[0]
        tr_sample_idxs = range(0,tr_sample_count)

        # Batch_count 

        batch_count = tr_sample_count / batch_size
        if tr_sample_count % batch_size:
            batch_count = batch_count+1

        # Yield X,y(batch_size), every for loop 
        for i in range(0, int(batch_count)):
            if i == batch_count-1:
                batch_size_idxs = tr_sample_idxs[i*batch_size : ]
            else : 
                batch_size_idxs = tr_sample_idxs[i*batch_size : (i+1)*batch_size]

            batch_size_idxs = sorted(batch_size_idxs)
            X = tr_data['X_data'][batch_size_idxs].reshape(len(batch_size_idxs),68,95,79)
            y = tr_data['y_data'][batch_size_idxs]

            yield X,y

In [24]:
mini_batch_size = 32
data_file_name = '180404_social_physical_masked_cross.hdf5'
label_to_class_name = [str(i) for i in range(2)]
selection_index_LRP = 0

## Model
The next part trains and evaluates a CNN.

In [30]:
def make_custom_model_cnn_2D():
       

    model = Sequential() #0
    model.add(Conv2D(8, (3,3), kernel_initializer='he_normal', padding='same', input_shape=(68,95,79)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))


    model.add(Conv2D(16, (3,3), kernel_initializer='he_normal', padding='same'))#4
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(32, (3,3), kernel_initializer='he_normal', padding='same'))#7
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(64, (3,3), kernel_initializer='he_normal', padding='same')) #10
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))


    model.add(Flatten()) #13
    model.add(Dense(128, kernel_initializer='he_normal'))
    model.add(Activation('relu'))
    
    model.add(Dense(2, kernel_initializer='he_normal'))
    model.add(Activation('linear'))
    
    model.add(Activation('softmax'))

#     model.add(Dense(1, activation='softmax', kernel_initializer='he_normal'))
    
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

    return model

In [33]:
# Create & train model
def train_model():

    """
    train model with data in every cross_i. Everytime model trained, get_result and generate_sequences are called
    """
    # Option : when you don't have enough GPUs
#     with tf.device('/cpu:0'):

    with h5py.File(data_file_name, "r") as data:

        for i in range(0,3):  # i is the cross number 59

            print('this is ith iter : ' , i)

            tr_data_X_name = 'cross_'+str(i+1)+ '_X'+'_train'
            tr_data_y_name = 'cross_'+str(i+1)+ '_y'+'_train'
            te_data_X_name = 'cross_'+str(i+1)+ '_X'+'_test'
            te_data_y_name = 'cross_'+str(i+1)+ '_y'+'_test'

            tr_data = {}
            tr_data['X_data'] = np.array(data[tr_data_X_name])
            tr_data['y_data'] = np.array(data[tr_data_y_name])
            
            te_data = {}
            te_data['X_data'] = np.array(data[te_data_X_name])
            te_data['y_data'] = np.array(data[te_data_y_name])
        
      

            
            tr_data['y_data'] = keras.utils.to_categorical(tr_data['y_data'], 2)
            te_data['y_data'] = keras.utils.to_categorical(te_data['y_data'], 2)
   
            training_sample_count = tr_data['X_data'].shape[0]


            # Initialize model

            model = make_custom_model_cnn_2D()
            model.compile(loss="categorical_crossentropy",
                          optimizer=Adam(),
                          metrics=["accuracy"])
            history = model.fit(tr_data['X_data'],tr_data['y_data'],
                                batch_size=mini_batch_size,
                                epochs=20,
                                verbose=1,shuffle=True)
            score = model.evaluate(te_data['X_data'], te_data['y_data'], verbose=0)
            return model,score


In [34]:

model,score = train_model()

this is ith iter :  0
Epoch 1/20
64/64 [==============================] - 1s 17ms/step - loss: 1.5238 - acc: 0.4375
Epoch 2/20
64/64 [==============================] - 0s 4ms/step - loss: 0.7527 - acc: 0.6250
Epoch 3/20
64/64 [==============================] - 0s 5ms/step - loss: 0.7641 - acc: 0.5312
Epoch 4/20
64/64 [==============================] - 0s 5ms/step - loss: 0.5956 - acc: 0.5625
Epoch 5/20
64/64 [==============================] - 0s 5ms/step - loss: 0.4485 - acc: 0.8906
Epoch 6/20
64/64 [==============================] - 0s 5ms/step - loss: 0.4789 - acc: 0.7188
Epoch 7/20
64/64 [==============================] - 0s 5ms/step - loss: 0.4154 - acc: 0.8594
Epoch 8/20
64/64 [==============================] - 0s 4ms/step - loss: 0.3277 - acc: 0.9844
Epoch 9/20
64/64 [==============================] - 0s 4ms/step - loss: 0.3083 - acc: 0.9844
Epoch 10/20
64/64 [==============================] - 0s 4ms/step - loss: 0.2508 - acc: 0.9844
Epoch 11/20
64/64 [===========================

## Accuracy

In [35]:

val_acc = []
with h5py.File(data_file_name, "r") as data:

    for i in range(0,3):  # i is the cross number 59

        print('this is ith iter : ' , i)

        te_data_X_name = 'cross_'+str(i+1)+ '_X'+'_test'
        te_data_y_name = 'cross_'+str(i+1)+ '_y'+'_test'

        te_data = {}
        te_data['X_data'] = np.array(data[te_data_X_name])
        te_data['y_data'] = np.array(data[te_data_y_name])
        
        te_data['y_data'] = keras.utils.to_categorical(te_data['y_data'], 2)
        

        score = model.evaluate(te_data['X_data'], te_data['y_data'], verbose=0)
        print(score[1])
        val_acc.append(score[1])

print("mean of validation accuracy",np.mean(val_acc))


this is ith iter :  0
0.75
this is ith iter :  1
1.0
this is ith iter :  2
1.0
mean of validation accuracy 0.9166666666666666


## Analysis

Next, we will set up a list of analysis methods by preparing tuples containing the methods' string identifiers used by `innvestigate.analyzer.create_analyzer(...)`, some optional parameters, a post processing choice for visualizing the computed analysis and a title for the figure to render. Analyzers can be deactivated by simply commenting the corresponding lines, or added by creating a new tuple as below.

For a full list of methods refer to the dictionary `investigate.analyzer.analyzers`

In [27]:
# Configure analysis methods and properties
if selection_index_LRP == None:
    methods = [
    # NAME                    OPT.PARAMS                POSTPROC FXN               TITLE

    # Show input
    ("lrp.epsilon",           {"epsilon": 1,"neuron_selection_mode":"max_activation"},           ivis.heatmap,        "LRP-Epsilon"),
    ]
else:    
    methods = [
        # NAME                    OPT.PARAMS                POSTPROC FXN               TITLE

        # Show input
        ("lrp.epsilon",           {"epsilon": 1, "neuron_selection_mode" : "index"},           ivis.heatmap,        "LRP-Epsilon"),
    ]

The main loop below will now instantiate the analyzer objects based on the loaded/trained model and the analyzers' parameterizations above.

In [20]:
# Create model without trailing softmax
model_wo_softmax = iutils.keras.graph.model_wo_softmax(model)

# Create analyzers.
analyzers = []
for method in methods:
    
    analyzer = innvestigate.create_analyzer(method[0],        # analysis method identifier
                                            model_wo_softmax, # model without softmax output
                                            **method[1])      # optional analysis parameters

    # Some analyzers require training.

    analyzer.fit(data, batch_size=256, verbose=1)
    analyzers.append(analyzer)
    

Now we analyze each image with the different analyzers:

In [29]:
# %%capture

with h5py.File(data_file_name, "r") as data:

    for i in range(0,1):  # i is the cross number 59

        print('this is ith iter : ' , i)

        tr_data_X_name = 'cross_'+str(i+1)+ '_X'+'_test'
        tr_data_y_name = 'cross_'+str(i+1)+ '_y'+'_test'

        tr_data = {}
        tr_data['X_data'] = np.array(data[tr_data_X_name])
        tr_data['y_data'] = np.array(data[tr_data_y_name])
        
        test_data = tr_data['X_data']
        test_label = tr_data['y_data']

        n = 10
        test_images = list(zip(test_data[:n], test_label[:n]))

        analysis = np.zeros([len(test_images), len(analyzers), 68, 95, 3])
        text = []
        R=[]



        for p, (x, y) in enumerate(test_images):
            # Add batch axis.
            x = x[None, :, :, :]

            # Predict final activations, probabilites, and label.
            presm = model_wo_softmax.predict_on_batch(x)[0]
            prob = model.predict_on_batch(x)[0]
            y_hat = prob.argmax()


            # Save prediction info:
            text.append(("ground truth label : %s" % label_to_class_name[int(y)],    # ground truth label
                         "pre-softmax logits : %.2f" % presm.max(),             # pre-softmax logits
                         "probabilistic softmax output   : %.2f" % prob.max(),              # probabilistic softmax output  
                         "predicted label : %s" % label_to_class_name[y_hat] # predicted label
                        ))

            for aidx, analyzer in enumerate(analyzers):
                # Analyze.
                if selection_index_LRP == None:
                    a = analyzer.analyze(x)
                else:
                    a = analyzer.analyze(x, neuron_selection=selection_index_LRP)
                R.append(a) # Save natual R
                
                """
                # Apply common postprocessing, e.g., re-ordering the channels for plotting.
                a = mnistutils.postprocess(a)
                # Apply analysis postprocessing, e.g., creating a heatmap.
                a = methods[aidx][2](a)
                # Store the analysis.
                analysis[i, aidx] = a[0]
                """


        # Save Relevance as
        R_name = 'social_physical_brain_relevance_cross_'+str(i)+'.npy'
        np.save(R_name, R)
        print(text)

this is ith iter :  0
[('ground truth label : 0', 'pre-softmax logits : 1.95', 'probabilistic softmax output   : 1.00', 'predicted label : 0'), ('ground truth label : 0', 'pre-softmax logits : 4.14', 'probabilistic softmax output   : 1.00', 'predicted label : 0'), ('ground truth label : 0', 'pre-softmax logits : 2.37', 'probabilistic softmax output   : 1.00', 'predicted label : 0'), ('ground truth label : 0', 'pre-softmax logits : 2.42', 'probabilistic softmax output   : 1.00', 'predicted label : 0'), ('ground truth label : 0', 'pre-softmax logits : -1.53', 'probabilistic softmax output   : 0.73', 'predicted label : 0'), ('ground truth label : 0', 'pre-softmax logits : 1.61', 'probabilistic softmax output   : 1.00', 'predicted label : 0'), ('ground truth label : 0', 'pre-softmax logits : 0.72', 'probabilistic softmax output   : 0.81', 'predicted label : 0'), ('ground truth label : 0', 'pre-softmax logits : 1.43', 'probabilistic softmax output   : 0.91', 'predicted label : 0'), ('ground